<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/TEC_OUR_Amp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import  drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
nltk.download('all')
from nltk.corpus import sentiwordnet as swn
device_name = tf.test.gpu_device_name()
print(device_name)
df = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/InfluenceAnalysis/TEC/TECOur.csv')
df = df.sample(frac=1,random_state=32)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [4]:
df.head()

,tweets,emotion
19554,smoothies are the best at any time any temper...,2
9801,Rain tomorrow that means curly hair,0
13413,Bout to lay it down im tired Tmrws gonna ...,0
3398,hahaha run a marathon hopin to go abroad to ...,0
9536,Christmas shopping in tkmax with Deck The Hall...,0


In [5]:
df.emotion.unique()

array([2, 0, 1, 3, 4, 5])

In [6]:
no_labels=6
category_dict = {}
for i in range(no_labels):
  category_dict[i]=[0 for j in range(no_labels)]
  category_dict[i][i]=1
data_tweet = [str(x).lower().split() for x in df['tweets']]
data_cat = np.array([np.array(category_dict[x]) for x in df['emotion']])
print(data_cat[:5])

[[0 0 1 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]]


In [7]:
print(data_tweet[:5])
print(data_cat[:5])

[['smoothies', 'are', 'the', 'best', 'at', 'any', 'time', 'any', 'temperature', 'we', 'have', 'no', 'smoothie', 'availability', 'at', 'lehigh'], ['rain', 'tomorrow', 'that', 'means', 'curly', 'hair'], ['bout', 'to', 'lay', 'it', 'down', 'im', 'tired', 'tmrws', 'gonna', 'feel', 'so', 'long', 'considering', 'i', 'have', 'not', 'had', 'school', 'for', 'a', 'week'], ['hahaha', 'run', 'a', 'marathon', 'hopin', 'to', 'go', 'abroad', 'to', 'do', 'some'], ['christmas', 'shopping', 'in', 'tkmax', 'with', 'deck', 'the', 'halls', 'playing', 'in', 'the', 'background']]
[[0 0 1 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]
 [1 0 0 0 0 0]]


In [8]:
# Parameters
Min_count = 0
Embedding_size = 200
Window_size = 5
Negative_sampling = 00

In [9]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)
w2v_sg.wv.init_sims(True)
w2v_cbow.wv.init_sims(True)

In [10]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/InfluenceAnalysis/glove/glove.twitter.27B.200d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [11]:
batch_size = 500
Max_input_size = max([len(x) for x in data_tweet])
data_tweet = [nltk.pos_tag(x) for x in data_tweet]
def getsent(word,tag):
  res=0
  try:
    x = swn.senti_synset(word+'.'+tag[0].lower()+'.01')
    res =  (x.pos_score()-x.neg_score())
  finally:
    return res 
data_tweet = [[(i[0],i[1],getsent(i[0],i[1])) for i in x] for x in data_tweet]

In [12]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=2):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(w2v.wv.word_vec(i[0]),i[1]) for i in x if i[0] in vocab]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ' or tweet[i][1][:2]=='RB':
            tweet[i]=((tweet[i][0])*mul_factor,tweet[i][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = np.array(data_cat[k*(batch_size):(k+1)*(batch_size)])
      yield (temp,tempres)          
      
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=2):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ' or tweet[i][1][:2]=='RB':
            tweet[i]=((tweet[i][0])*mul_factor,tweet[i][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = np.array(data_cat[k*(batch_size):(k+1)*(batch_size)])
      yield (temp,tempres)

In [13]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(data_tweet, data_cat)
X_train=[]
X_test=[]
y_train=[]
y_test=[]
counter=0
for train_index, test_index in sss.split(data_tweet, data_cat):
  print("TRAIN:", train_index, "TEST:", test_index)
  for i in train_index:
    X_train+=[data_tweet[i]]
    y_train+=[data_cat[i]]

  for j in test_index:
    X_test+=[data_tweet[j]]
    y_test+=[data_cat[j]]

 # X_train, X_test = data_tweet[train_index], data_tweet[test_index]
 # y_train, y_test = data_cat[train_index], data_cat[test_index]
print(X_train[:5])
print(y_train[:5])
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(X_train, y_train)
x_train=[]
X_val=[]
Y_train=[]
y_val=[]
for train_index1, test_index1 in sss.split(X_train, y_train):
  print("TRAIN:", train_index1, "TEST:", test_index1)
  for i in train_index1:
    x_train+=[X_train[i]]
    Y_train+=[y_train[i]]

  for j in test_index1:
    X_val+=[X_train[j]]
    y_val+=[y_train[j]]
X_train=x_train
y_train=Y_train
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))
print(len(X_val))
print(len(y_val))
print(X_train[:5])
print(y_train[:5])
result_table = [0,0,0]

TRAIN: [ 4990  8232 16430 ...  1782 14725 11582] TEST: [ 9096 15406 12733 ... 11184  9511 11639]
[[('what', 'WP', 0), ('the', 'DT', 0), ('hell', 'NN', -0.375), ('has', 'VBZ', 0), ('gotten', 'VBN', 0), ('in', 'IN', 0), ('to', 'TO', 0), ('you', 'PRP', 0), ('like', 'IN', 0), ('legit', 'VB', 0), ('your', 'PRP$', 0), ('an', 'DT', 0), ('ass', 'NN', 0.0), ('now', 'RB', 0.0)], [('i', 'NNS', 0.0), ('do', 'VBP', 0.0), ('not', 'RB', -0.625), ('have', 'VB', 0.25), ('cell', 'NN', 0.0), ('phone', 'NN', 0.0), ('service', 'NN', 0.0), ('so', 'RB', 0.0), ('i', 'JJ', 0), ('am', 'VBP', 0), ('tweeting', 'VBG', 0), ('you', 'PRP', 0), ('are', 'VBP', 0), ('you', 'PRP', 0), ('still', 'RB', -0.125), ('down', 'RP', -0.125), ('for', 'IN', 0), ('some', 'DT', 0), ('shopping', 'NN', 0.0)], [('there', 'EX', 0), ('is', 'VBZ', 0), ('a', 'DT', 0), ('mysterious', 'JJ', 0), ('ticking', 'VBG', 0), ('noise', 'NN', 0.0), ('somewhere', 'RB', 0.0), ('by', 'IN', 0), ('my', 'PRP$', 0), ('bed', 'NN', 0.0), ('not', 'RB', -0.625), 

In [14]:
epochs = 30

In [15]:
from keras import backend as K
def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recalls = true_positives / (possible_positives + K.epsilon())
        return recalls
def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precisions = true_positives / (predicted_positives + K.epsilon())
        return precisions
def f1(y_true, y_pred):
    precisions = precision(y_true, y_pred)
    recalls = recall(y_true, y_pred)
    return 2*((precisions*recalls)/(precisions+recalls+K.epsilon()))

In [16]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dropout(0.2))
model_sg_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg_bi.add(tf.keras.layers.Dropout(0.2))
model_sg_bi.add(tf.keras.layers.Dense(6, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
sg_train_gen=vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size)
sg_val_gen=vec_gen(w2v_sg,vocab_sg,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_sg_bi = model_sg_bi.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)
result_table[2]=history[1]
print("SG_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
27/27 [==============================] - 52s 2s/step - loss: 1.7371 - accuracy: 0.3182 - f1: 0.0103 - precision: 0.0484 - recall: 0.0064 - val_loss: 1.5587 - val_accuracy: 0.3877 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/30
27/27 [==============================] - 39s 1s/step - loss: 1.5472 - accuracy: 0.4000 - f1: 0.0592 - precision: 0.5973 - recall: 0.0318 - val_loss: 1.5018 - val_accuracy: 0.4126 - val_f1: 0.1654 - val_precision: 0.6003 - val_recall: 0.0960
Epoch 3/30
27/27 [==============================] - 39s 1s/step - loss: 1.4946 - accuracy: 0.4166 - f1: 0.2035 - precision: 0.6360 - recall: 0.1242 - val_loss: 1.4434 - val_accuracy: 0.4531 - val_f1: 0.2363 - val_precision: 0.6914 - val_recall: 0.1426
Epoch 4/30
27/27 [==============================] - 39s 1s/step - loss: 1.4305 - accuracy: 0.4469 - f1: 0.2686 - precision: 0.6603 - recall: 0.1718 - val_loss: 1.3832 - val_accuracy: 0.4726 - val_f1: 0.3146 - val_precision: 0.7152 -

In [17]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dropout(0.2))
model_cbow_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_cbow_bi.add(tf.keras.layers.Dropout(0.2))
model_cbow_bi.add(tf.keras.layers.Dense(6, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
cbow_train_gen=vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size)
cbow_val_gen=vec_gen(w2v_cbow,vocab_cbow,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_cbow_bi = model_cbow_bi.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("CBOW_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
27/27 [==============================] - 44s 2s/step - loss: 1.7172 - accuracy: 0.3331 - f1: 0.0182 - precision: 0.0711 - recall: 0.0126 - val_loss: 1.5572 - val_accuracy: 0.3877 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/30
27/27 [==============================] - 42s 2s/step - loss: 1.5434 - accuracy: 0.3991 - f1: 0.0680 - precision: 0.5730 - recall: 0.0375 - val_loss: 1.5104 - val_accuracy: 0.4080 - val_f1: 0.1611 - val_precision: 0.6085 - val_recall: 0.0929
Epoch 3/30
27/27 [==============================] - 39s 1s/step - loss: 1.5045 - accuracy: 0.4093 - f1: 0.1797 - precision: 0.5902 - recall: 0.1069 - val_loss: 1.4708 - val_accuracy: 0.4291 - val_f1: 0.2285 - val_precision: 0.6672 - val_recall: 0.1380
Epoch 4/30
27/27 [==============================] - 40s 2s/step - loss: 1.4484 - accuracy: 0.4297 - f1: 0.2585 - precision: 0.6531 - recall: 0.1635 - val_loss: 1.4246 - val_accuracy: 0.4511 - val_f1: 0.2396 - val_precision: 0.7081 -

In [18]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dropout(0.2))
model_glove_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_glove_bi.add(tf.keras.layers.Dropout(0.2))
model_glove_bi.add(tf.keras.layers.Dense(6, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
glove_train_gen=glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size)
glove_val_gen=glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_glove_bi = model_glove_bi.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("GLOVE_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
27/27 [==============================] - 22s 695ms/step - loss: 1.6355 - accuracy: 0.3779 - f1: 0.0440 - precision: 0.2746 - recall: 0.0246 - val_loss: 1.4291 - val_accuracy: 0.4397 - val_f1: 0.2810 - val_precision: 0.6868 - val_recall: 0.1769
Epoch 2/30
27/27 [==============================] - 18s 662ms/step - loss: 1.3811 - accuracy: 0.4693 - f1: 0.3429 - precision: 0.6820 - recall: 0.2303 - val_loss: 1.2561 - val_accuracy: 0.5131 - val_f1: 0.4201 - val_precision: 0.7329 - val_recall: 0.2949
Epoch 3/30
27/27 [==============================] - 18s 664ms/step - loss: 1.2473 - accuracy: 0.5275 - f1: 0.4273 - precision: 0.7167 - recall: 0.3052 - val_loss: 1.1765 - val_accuracy: 0.5594 - val_f1: 0.5075 - val_precision: 0.7324 - val_recall: 0.3886
Epoch 4/30
27/27 [==============================] - 18s 671ms/step - loss: 1.1508 - accuracy: 0.5692 - f1: 0.5009 - precision: 0.7294 - recall: 0.3816 - val_loss: 1.1547 - val_accuracy: 0.5546 - val_f1: 0.5153 - val_precision: 0.7402 -

In [19]:
result_table[0]=history[1]
print("GLOVE_BI")

GLOVE_BI


In [20]:
#print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("glove",result_table[:1])
print("cbow",result_table[1:2])
print("sg",result_table[2:3])
# print(history_sg_bi.history)
# plt.title("Skip-Gram + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_sg_bi.history['loss'],c='b',label='loss')
# plt.plot(history_sg_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()
# plt.title("GloVe + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_glove_bi.history['loss'],c='b',label='loss')
# plt.plot(history_glove_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()

glove [0.5582500100135803]
cbow [0.4634999930858612]
sg [0.47975000739097595]
